In [24]:
import requests
import time
from requests.compat import urljoin
from bs4 import BeautifulSoup
from PIL import Image
import io
import shutil
import os

# Load all Images

In [25]:
dirname = "data"
ims = []
for filename in os.listdir(dirname) :
    fullname = os.path.join(dirname, filename)
    #print(fullname)
    im = Image.open(fullname)
    #display(im)
    ims.append(im)
print("images : " + str(len(ims)))

images : 167


# Split to single Digits 1/2/3/4

In [26]:
print("image size : " + str(ims[0].size))

rects = [(17,12,35,35),(37,10,53,33),(56,10,72,33),(76,12,92,35)]

digit1 = []
digit2 = []
digit3 = []
digit4 = []

for im in ims:
    # split first digit
    digit1.append(im.copy().crop(rects[0]))
    digit2.append(im.copy().crop(rects[1]))
    digit3.append(im.copy().crop(rects[2]))
    digit4.append(im.copy().crop(rects[3]))
    #display(digit1[-1])

image size : (100, 40)


# Convert RGB -> Grey and make np-Array

In [30]:
import numpy as np

digit1[-1].getbands()

np_digit1 = []
for im in digit1:
    np_digit1.append( np.array(im.convert("L").getdata()) )
#np_digit1[-1]

np_digit2 = []
for im in digit2:
    np_digit2.append( np.array(im.convert("L").getdata()) )
#np_digit2[-1]

np_digit3 = []
for im in digit3:
    np_digit3.append( np.array(im.convert("L").getdata()) )
#np_digit1[-1]

np_digit4 = []
for im in digit4:
    np_digit4.append( np.array(im.convert("L").getdata()) )
#np_digit1[-1]


# Cluster Images

In [31]:
from sklearn.cluster import KMeans

#20 clusters : 0...9 / "upware bent","downward bent"

kmeans1 = KMeans(n_clusters=20,random_state=22)
kmeans1.fit(np_digit1)

kmeans2 = KMeans(n_clusters=20,random_state=22)
kmeans2.fit(np_digit2)

kmeans3 = KMeans(n_clusters=20,random_state=22)
kmeans3.fit(np_digit3)

kmeans4 = KMeans(n_clusters=20,random_state=22)
kmeans4.fit(np_digit4)

KMeans(n_clusters=20, random_state=22)

# Show Clusters

In [36]:
#print(kmeans1.labels_)
groups1 = {}
for file, cluster in zip(digit1,kmeans1.labels_):
    if cluster not in groups1.keys():
        groups1[cluster] = []
        groups1[cluster].append(file)
    else:
        groups1[cluster].append(file)
        
for g in groups1:
    group_ims = groups1[g]
    l = len(group_ims)
    #print(g, l)
    width = group_ims[0].size[0]
    im_concat = Image.new("L", (width * l, group_ims[0].size[1]))
    offset=0
    for im in group_ims:
        im_concat.paste(im, (offset,0))
        offset += width
    display(im_concat)
print()

#

#print(kmeans2.labels_)
groups2 = {}
for file, cluster in zip(digit2,kmeans2.labels_):
    if cluster not in groups2.keys():
        groups2[cluster] = []
        groups2[cluster].append(file)
    else:
        groups2[cluster].append(file)

for g in groups2:
    group_ims = groups2[g]
    l = len(group_ims)
    #print(g, l)
    width = group_ims[0].size[0]
    im_concat = Image.new("L", (width * l, group_ims[0].size[1]))
    offset=0
    for im in group_ims:
        im_concat.paste(im, (offset,0))
        offset += width
    display(im_concat)
print()

#

#print(kmeans2.labels_)
groups3 = {}
for file, cluster in zip(digit3,kmeans3.labels_):
    if cluster not in groups3.keys():
        groups3[cluster] = []
        groups3[cluster].append(file)
    else:
        groups3[cluster].append(file)

for g in groups3:
    group_ims = groups3[g]
    l = len(group_ims)
    #print(g, l)
    width = group_ims[0].size[0]
    im_concat = Image.new("L", (width * l, group_ims[0].size[1]))
    offset=0
    for im in group_ims:
        im_concat.paste(im, (offset,0))
        offset += width
    display(im_concat)
print()

#

#print(kmeans2.labels_)
groups4 = {}
for file, cluster in zip(digit4,kmeans4.labels_):
    if cluster not in groups4.keys():
        groups4[cluster] = []
        groups4[cluster].append(file)
    else:
        groups4[cluster].append(file)

for g in groups4:
    group_ims = groups4[g]
    l = len(group_ims)
    #print(g, l)
    width = group_ims[0].size[0]
    im_concat = Image.new("L", (width * l, group_ims[0].size[1]))
    offset=0
    for im in group_ims:
        im_concat.paste(im, (offset,0))
        offset += width
    display(im_concat)
print()